# CARREGAMENTO DO DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# CÉLULA 2 — Importação de bibliotecas e definição do caminho do arquivo

Esta célula:

Instala e importa as bibliotecas necessárias.

Define o caminho do seu arquivo PDF no Drive.

Lê as metainformações (número total de páginas e alguns metadados básicos).

In [2]:
# CÉLULA 2 — Importação e leitura inicial do PDF

!pip install pymupdf tqdm --quiet

import fitz  # PyMuPDF
from tqdm import tqdm
import os

# Caminho do arquivo PDF no seu Google Drive
pdf_path = "/content/drive/MyDrive/handbook-of-Ferroalloys.pdf"

# Verifica se o arquivo existe
if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"O arquivo não foi encontrado: {pdf_path}")

# Abre o PDF
doc = fitz.open(pdf_path)

# Mostra informações básicas
print(f"✅ PDF carregado com sucesso!")
print(f"📄 Total de páginas: {len(doc)}")
print(f"📘 Metadados: {doc.metadata}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 98.8 MB/s eta 0:00:00
✅ PDF carregado com sucesso!
📄 Total de páginas: 507
📘 Metadados: {'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'PDF Compressor 2019 / WinSoftMagic.Com', 'creationDate': "D:20161104212544+05'30'", 'modDate': "D:20250929155107-03'00'", 'trapped': '', 'encryption': None}


In [3]:
# CÉLULA 3 — Detecção de capítulos (TOC + heurística de títulos)

import re
import numpy as np
from tqdm import tqdm
import fitz  # PyMuPDF
import os

# Reutiliza o 'doc' e 'pdf_path' da CÉLULA 2. Se não existirem, abre de novo.
try:
    doc
except NameError:
    pdf_path = "/content/drive/MyDrive/handbook-of-Ferroalloys.pdf"
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {pdf_path}")
    doc = fitz.open(pdf_path)

N_PAGES = len(doc)

def normalize_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def uppercase_ratio(s: str) -> float:
    letters = [ch for ch in s if ch.isalpha()]
    if not letters:
        return 0.0
    upp = sum(1 for ch in letters if ch.isupper())
    return upp / len(letters)

def extract_biggest_span_title(page):
    """
    Retorna (title, max_font_size, upper_ratio) extraindo o maior 'span' de texto da página.
    Se a página for essencialmente imagem ou vazia, retorna (None, 0, 0).
    """
    try:
        d = page.get_text("dict")
    except Exception:
        return (None, 0, 0)

    best_span = None
    best_size = 0.0

    # Varre blocos/linhas/spans e pega o span com maior 'size'
    for b in d.get("blocks", []):
        if b.get("type", 0) != 0:
            continue
        for l in b.get("lines", []):
            for s in l.get("spans", []):
                size = s.get("size", 0) or 0
                text = s.get("text", "") or ""
                txt = normalize_spaces(text)
                if not txt:
                    continue
                # Evita lixo muito curto ou headings numéricos puros
                if len(txt) < 3:
                    continue
                if size > best_size:
                    best_size = size
                    best_span = txt

    if not best_span:
        return (None, 0.0, 0.0)

    ur = uppercase_ratio(best_span)
    return (best_span, float(best_size), ur)

# 1) Tenta usar TOC nativo (sumário)
toc = []
try:
    toc = doc.get_toc()  # lista de [level, title, page_num]
except Exception:
    toc = []

toc_entries = []
if toc:
    for level, title, page_num in toc:
        # PyMuPDF TOC usa 1-based p/ páginas
        title = normalize_spaces(title)
        if title:
            toc_entries.append({"level": level, "title": title, "page": int(page_num)})

# 2) Heurística por página (apenas se necessário, mas coletamos para comparação)
candidates = []
font_sizes = []

for i in tqdm(range(N_PAGES), desc="Varredura de páginas p/ títulos"):
    page = doc.load_page(i)
    title, fsize, ur = extract_biggest_span_title(page)
    font_sizes.append(fsize)
    candidates.append({
        "page": i + 1,                # 1-based
        "title": title,
        "font_size": fsize,
        "upper_ratio": ur,
    })

# Determina limiar de "título grande" (percentil 92, por ex.)
fs_arr = np.array(font_sizes, dtype=float)
fs_thresh = float(np.nanpercentile(fs_arr[fs_arr > 0], 92.0)) if np.any(fs_arr > 0) else 0.0

# Palavras-chave típicas de capítulos/seções de handbook
title_keywords = re.compile(
    r"^(chapter|cap[ií]tulo|section|appendix|preface|foreword|introduction|abstract|glossary|references|contents|table of contents)\b",
    re.IGNORECASE
)

# 3) Sugestão de páginas-início:
#    - Se houver TOC, usamos suas páginas.
#    - Caso não, usamos mudanças de "título provável" por:
#        (a) fonte acima do limiar OU (b) match de palavra-chave,
#        com título diferente do anterior e upper_ratio razoável.
suggested_starts = []

if toc_entries:
    # Usa TOC diretamente
    for entry in toc_entries:
        pg = max(1, min(N_PAGES, entry["page"]))
        suggested_starts.append(pg)
    # Remove duplicatas e ordena
    suggested_starts = sorted(sorted(set(suggested_starts)))
else:
    last_title_key = None
    for c in candidates:
        t = (c["title"] or "").strip()
        if not t:
            continue

        is_kw = bool(title_keywords.search(t))
        is_big = c["font_size"] >= fs_thresh if fs_thresh > 0 else False
        is_upperish = c["upper_ratio"] >= 0.60

        # "Chave" de título para evitar repetição por rodapé/cabeçalho
        # Mantém apenas as primeiras 80 chars e remove números isolados no fim
        t_key = re.sub(r"\b\d+\b$", "", t.lower()).strip()[:80]

        # Critério: título grande ou keyword, com alguma "força" visual
        if (is_kw or is_big or (is_kw and is_upperish)):
            if not last_title_key or t_key != last_title_key:
                suggested_starts.append(c["page"])
                last_title_key = t_key

    # Garantias mínimas: inclui página 1 se nada foi detectado
    if 1 not in suggested_starts:
        suggested_starts.insert(0, 1)

    # Ordena e limpa duplicatas
    suggested_starts = sorted(sorted(set(suggested_starts)))

# --- Saídas para validação ---
print("\n===== RESUMO DA DETECÇÃO =====")
print(f"Total de páginas: {N_PAGES}")
if toc_entries:
    print(f"TOC encontrado: {len(toc_entries)} entradas")
    # Mostra até 25 entradas do TOC para inspeção
    for e in toc_entries[:25]:
        print(f"- p.{e['page']:>4} | L{e['level']} | {e['title']}")
    if len(toc_entries) > 25:
        print(f"... (+{len(toc_entries)-25} entradas)")
else:
    print("TOC não encontrado ou vazio. Usando heurística de títulos.")
    print(f"Limiar de fonte (p92): {fs_thresh:.2f}")
    print("Amostra de candidatos (até 25):")
    shown = 0
    for c in candidates:
        if c["title"] and shown < 25:
            print(f"- p.{c['page']:>4} | size={c['font_size']:.1f} | upper={c['upper_ratio']:.2f} | {c['title'][:120]}")
            shown += 1

print("\n===== PÁGINAS SUGERIDAS COMO INÍCIO DE CAPÍTULO =====")
print(suggested_starts)

# Também devolve como variável para próxima célula
chapter_starts = suggested_starts


Varredura de páginas p/ títulos: 100%|██████████| 507/507 [00:02<00:00, 169.60it/s]


===== RESUMO DA DETECÇÃO =====
Total de páginas: 507
TOC encontrado: 357 entradas
- p.   2 | L1 | 1. Introduction
- p.   2 | L2 | 1.1 Introduction to Ferroalloys
- p.   4 | L2 | 1.2 The Scope and Structure of the Book
- p.   6 | L2 | Acknowledgments
- p.   6 | L2 | References
- p.   7 | L1 | 2. Basics of Ferroalloys
- p.   7 | L2 | 2.1 Introduction: Background for Ferroalloys Development and Production
- p.  10 | L2 | 2.2 Ferroalloys in the Development of Steels
- p.  12 | L2 | 2.3 A Case: Ferroalloys for Stainless Steels
- p.  15 | L2 | 2.4 Recent Ferroalloys Production and Markets Development
- p.  20 | L2 | 2.5 Energy and Emissions Issues of the Ferroalloys Industry
- p.  20 | L3 | 2.5.1 Energy Demand for Ferroalloys Making
- p.  21 | L3 | 2.5.2 CO2 Emissions from Ferroalloys Production
- p.  24 | L2 | 2.6 Future Outlook for the Ferroalloys Industry
- p.  25 | L2 | References
- p.  27 | L1 | 3. Theory of Ferroalloys Processing
- p.  27 | L2 | 3.1 General Theory of the Processes of 

In [4]:
# CÉLULA 4 — Exportar capítulos (TOC nível 1)

import os
import re
from tqdm import tqdm
import fitz  # PyMuPDF

# Reutiliza 'doc' e 'pdf_path' das células anteriores; reabra se necessário.
try:
    doc
except NameError:
    pdf_path = "/content/drive/MyDrive/handbook-of-Ferroalloys.pdf"
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {pdf_path}")
    doc = fitz.open(pdf_path)

N_PAGES = len(doc)

# Saída
out_dir = "/content/drive/MyDrive/handbook_split_by_chapter"
os.makedirs(out_dir, exist_ok=True)

# Carrega TOC
toc = doc.get_toc(simple=False)  # lista de tuplas (level, title, page)
# Filtro pelos capítulos (level == 1)
chapters = []
for item in toc:
    # PyMuPDF retorna objetos do tipo (level, title, page) ou dicts dependendo da versão;
    # normalizamos:
    if isinstance(item, (list, tuple)):
        level, title, page = item[0], item[1], item[2]
    else:
        level, title, page = item.get("level"), item.get("title"), item.get("page")
    if not isinstance(level, int):
        continue
    if level == 1 and isinstance(title, str) and title.strip():
        chapters.append({"title": re.sub(r"\s+", " ", title).strip(), "page": int(page)})

if not chapters:
    raise RuntimeError("TOC presente, mas não há entradas de nível 1. Precisaremos cair no plano B (heurística).")

# Ordena por página e monta faixas
chapters = sorted(chapters, key=lambda x: x["page"])
# Garante que não haja páginas fora do documento
for ch in chapters:
    ch["page"] = max(1, min(N_PAGES, ch["page"]))

# Calcula intervalos [start, end] (1-based inclusivo)
ranges = []
for i, ch in enumerate(chapters):
    start = ch["page"]
    if i < len(chapters) - 1:
        end = chapters[i+1]["page"] - 1
    else:
        end = N_PAGES
    # saneamento
    if end < start:
        end = start
    ranges.append({"idx": i+1, "title": ch["title"], "start": start, "end": end})

# Função para nomear arquivos de forma segura
def sanitize_filename(name: str, maxlen=100) -> str:
    name = name.strip()
    name = re.sub(r"[\\/:*?\"<>|]+", " ", name)   # remove caracteres inválidos em FS
    name = re.sub(r"\s+", " ", name).strip()
    if len(name) > maxlen:
        name = name[:maxlen].rstrip()
    return name

# Mostra prévia dos capítulos detectados
print("Capítulos detectados (TOC nível 1):")
for r in ranges:
    print(f"- {r['idx']:02d}: p.{r['start']}–{r['end']} | {r['title']}")

# Exporta cada capítulo
exported_files = []
for r in tqdm(ranges, desc="Exportando capítulos"):
    start0 = r["start"] - 1  # zero-based
    end0   = r["end"] - 1    # zero-based

    # cria um novo documento e insere o intervalo
    newdoc = fitz.open()
    newdoc.insert_pdf(doc, from_page=start0, to_page=end0)

    fname = f"{r['idx']:02d} - {sanitize_filename(r['title'])}.pdf"
    fpath = os.path.join(out_dir, fname)
    newdoc.save(fpath, deflate=True, garbage=4)  # compacta um pouco
    newdoc.close()

    exported_files.append(fpath)

print("\nArquivos gerados:")
for p in exported_files:
    print("-", p)

print(f"\nConcluído. Saída em: {out_dir}")


Capítulos detectados (TOC nível 1):
- 01: p.2–6 | 1. Introduction
- 02: p.7–26 | 2. Basics of Ferroalloys
- 03: p.27–80 | 3. Theory of Ferroalloys Processing
- 04: p.81–136 | 4. Ferroalloys Processing Equipment
- 05: p.137–173 | 5. Electric and Thermal Operations of Furnaces for Ferroalloys Production
- 06: p.174–215 | 6. Ferrosilicon and Silicon Technology
- 07: p.216–261 | 7. Manganese Ferroalloys Technology
- 08: p.262–311 | 8. Technology of Chromium and Its Ferroalloys
- 09: p.312–358 | 9. High Carbon Ferrochrome Technology
- 10: p.359–367 | 10. Technology of Ferronickel
- 11: p.368–376 | 11. Technology of Tungsten Ferroalloys
- 12: p.377–386 | 12. Technology of Molybdenum Ferroalloys
- 13: p.387–399 | 13. Technology of Vanadium Ferroalloys
- 14: p.400–408 | 14. Technology of Niobium Ferroalloys
- 15: p.409–421 | 15. Technology of Titanium Ferroalloys
- 16: p.422–434 | 16. Technology of Zirconium Ferroalloys
- 17: p.435–443 | 17. Boron Ferroalloys
- 18: p.444–454 | 18. Technology o

Exportando capítulos: 100%|██████████| 26/26 [00:01<00:00, 20.76it/s]


Arquivos gerados:
- /content/drive/MyDrive/handbook_split_by_chapter/01 - 1. Introduction.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/02 - 2. Basics of Ferroalloys.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/03 - 3. Theory of Ferroalloys Processing.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/04 - 4. Ferroalloys Processing Equipment.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/05 - 5. Electric and Thermal Operations of Furnaces for Ferroalloys Production.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/06 - 6. Ferrosilicon and Silicon Technology.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/07 - 7. Manganese Ferroalloys Technology.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/08 - 8. Technology of Chromium and Its Ferroalloys.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/09 - 9. High Carbon Ferrochrome Technology.pdf
- /content/drive/MyDrive/handbook_split_by_chapter/10 - 10. Technology of Ferronickel.pdf
- /co

In [5]:
!ls /content/drive


MyDrive  Othercomputers  Shareddrives


In [6]:
# CÉLULA 8 — Mover PDFs de capítulos para a pasta bibliografia

import os
import shutil
from tqdm import tqdm

src_dir = "/content/drive/MyDrive/handbook_split_by_chapter"
dst_dir = "/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/bibliografia"

# Garante que o destino exista
os.makedirs(dst_dir, exist_ok=True)

files = [f for f in os.listdir(src_dir) if f.lower().endswith(".pdf")]
print(f"Total de arquivos a mover: {len(files)}")

for f in tqdm(files, desc="Movendo arquivos"):
    src_path = os.path.join(src_dir, f)
    dst_path = os.path.join(dst_dir, f)
    shutil.move(src_path, dst_path)

print(f"\n✅ Todos os arquivos foram movidos para:\n{dst_dir}")


Total de arquivos a mover: 26


Movendo arquivos: 100%|██████████| 26/26 [00:00<00:00, 254.84it/s]


✅ Todos os arquivos foram movidos para:
/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/bibliografia


In [7]:
# CÉLULA 9 — Auditoria dos PDFs em "bibliografia"

!pip install pymupdf pandas --quiet

import os, re, fitz, pandas as pd

base_dir = "/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/bibliografia"
SIZE_MB_LIMIT = 150  # ajuste se quiser outro limite para o NotebookLM

rows = []
for fname in sorted(os.listdir(base_dir)):
    if not fname.lower().endswith(".pdf"):
        continue
    fpath = os.path.join(base_dir, fname)
    size_mb = os.path.getsize(fpath) / (1024*1024)

    with fitz.open(fpath) as d:
        n_pages = len(d)

    m = re.match(r"^(\d{2})\s*-\s*(.+)\.pdf$", fname, flags=re.IGNORECASE)
    idx = int(m.group(1)) if m else None
    title = m.group(2) if m else os.path.splitext(fname)[0]

    rows.append({
        "idx": idx,
        "file": fname,
        "title": title,
        "pages": n_pages,
        "size_mb": round(size_mb, 2),
        "status": "GRANDE" if size_mb > SIZE_MB_LIMIT else "ok"
    })

df = pd.DataFrame(rows).sort_values(by=["idx","file"], na_position="last").reset_index(drop=True)
df


,idx,file,title,pages,size_mb,status
0,1,01 - 1. Introduction.pdf,1. Introduction,5,0.10,ok
1,2,02 - 2. Basics of Ferroalloys.pdf,2. Basics of Ferroalloys,20,0.18,ok
2,3,03 - 3. Theory of Ferroalloys Processing.pdf,3. Theory of Ferroalloys Processing,54,0.66,ok
3,4,04 - 4. Ferroalloys Processing Equipment.pdf,4. Ferroalloys Processing Equipment,56,0.73,ok
4,5,05 - 5. Electric and Thermal Operations of Fur...,5. Electric and Thermal Operations of Furnaces...,37,1.16,ok
5,6,06 - 6. Ferrosilicon and Silicon Technology.pdf,6. Ferrosilicon and Silicon Technology,42,0.40,ok
6,7,07 - 7. Manganese Ferroalloys Technology.pdf,7. Manganese Ferroalloys Technology,46,0.44,ok
7,8,08 - 8. Technology of Chromium and Its Ferroal...,8. Technology of Chromium and Its Ferroalloys,50,0.33,ok
8,9,09 - 9. High Carbon Ferrochrome Technology.pdf,9. High Carbon Ferrochrome Technology,47,0.66,ok
9,10,10 - 10. Technology of Ferronickel.pdf,10. Technology of Ferronickel,9,0.08,ok


In [8]:
# CÉLULA 10 — Manifesto (CSV + README.md)

import os, re, fitz, pandas as pd

base_dir = "/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/bibliografia"

rows = []
for fname in sorted(os.listdir(base_dir)):
    if not fname.lower().endswith(".pdf"):
        continue
    fpath = os.path.join(base_dir, fname)
    size_mb = os.path.getsize(fpath)/(1024*1024)
    with fitz.open(fpath) as d:
        n_pages = len(d)

    m = re.match(r"^(\d{2})\s*-\s*(.+)\.pdf$", fname, flags=re.IGNORECASE)
    idx = int(m.group(1)) if m else None
    title = m.group(2) if m else os.path.splitext(fname)[0]

    rows.append({
        "idx": idx,
        "file": fname,
        "title": title,
        "pages": n_pages,
        "size_mb": round(size_mb, 2)
    })

df = pd.DataFrame(rows).sort_values(by=["idx","file"], na_position="last").reset_index(drop=True)

csv_path = os.path.join(base_dir, "manifest_capitulos.csv")
md_path  = os.path.join(base_dir, "README.md")

df.to_csv(csv_path, index=False, encoding="utf-8")

# README.md simples com lista
lines = [
    "# Handbook of Ferroalloys — Capítulos divididos\n",
    "Arquivos prontos para uso no NotebookLM. Manifesto: `manifest_capitulos.csv`.\n",
    "| idx | páginas | tamanho (MB) | arquivo | título |",
    "|---:|---:|---:|---|---|",
]
for r in df.to_dict(orient="records"):
    lines.append(f"| {r['idx'] if r['idx'] is not None else ''} | {r['pages']} | {r['size_mb']} | {r['file']} | {r['title']} |")

with open(md_path, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print("Gerados:")
print("-", csv_path)
print("-", md_path)


Gerados:
- /content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/bibliografia/manifest_capitulos.csv
- /content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/bibliografia/README.md


# TRATAMENTO DE DADOS

In [11]:
# CÉLULA A1+ — DRY-RUN (nada salvo): inventário + n_cols/n_rows + cobertura temporal se houver timestamp

from pathlib import Path
import os, re, io, csv
from datetime import datetime
from difflib import SequenceMatcher
import pandas as pd

# Diretórios
ROOT = Path("/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados")
BASE_DIR = ROOT / "dados_iniciais"

# --- Heurísticas ---

# Fornos em qualquer posição: F1, F-1, F_1, F 1, Forno 4 etc.
FORNO_REGEXES = [
    re.compile(r"(?i)\bF\s*[-_ ]?\s*(\d{1,2})\b"),
    re.compile(r"(?i)F\s*[-_ ]?\s*(\d{1,2})"),
    re.compile(r"(?i)\bForno\s*[-_ ]?\s*(\d{1,2})\b")
]
def detect_fornos_anywhere(text: str):
    found = []
    for rgx in FORNO_REGEXES:
        for m in rgx.findall(text):
            num = str(m) if isinstance(m, (str,int)) else (m[0] if m else "")
            if num:
                tag = f"F{int(num)}"
                if tag not in found:
                    found.append(tag)
    return found

VAR_KEYWORDS = [
    (r"consum", "Consumo"),
    (r"corrid|corrida|run", "Corrida"),
    (r"granul|granulo", "Granulometria"),
    (r"umidad", "Umidade"),
    (r"poten|kw|power", "Potencia"),
    (r"corrente|amp|amper", "Corrente"),
    (r"tens[aã]o|volt", "Tensao"),
    (r"carg|charge|aliment", "Carga/Alimentacao"),
    (r"temp|temperat", "Temperatura"),
    (r"rendi|yield", "Rendimento"),
    (r"eletrod|compriment|eletrodo", "Comprimento_Eletrodo"),
    (r"supervis", "Supervisorio"),
    (r"\bf4\b", "F4_Supervisorio"),
    (r"\bf5\b", "F5_Supervisorio"),
    (r"dicion|data[_-]?dict|dic\b", "Dicionario_Dados"),
]
def detect_variable_guess(text: str):
    t = text.lower()
    hits = []
    for pat, label in VAR_KEYWORDS:
        if re.search(pat, t):
            hits.append(label)
    if not hits:
        return ""
    order = ["F4_Supervisorio","F5_Supervisorio","Supervisorio","Comprimento_Eletrodo",
             "Consumo","Corrida","Temperatura","Potencia","Corrente","Tensao",
             "Carga/Alimentacao","Granulometria","Umidade","Rendimento","Dicionario_Dados"]
    for lbl in order:
        if lbl in hits:
            return lbl
    return hits[0]

def normalize_name_for_match(name: str):
    base = os.path.splitext(name)[0].lower()
    base = re.sub(r"[\W_]+", " ", base)
    base = re.sub(r"\b(v\d+|vers[aã]o?\d*|final|novo|corrigido)\b", " ", base)
    base = re.sub(r"\s+", " ", base).strip()
    return base

def similarity(a: str, b: str) -> float:
    return SequenceMatcher(None, a, b).ratio()

# Timestamp detection helpers
TS_COL_HINTS = re.compile(r"(date|data|time|hora|timestamp|dt|datetime)", re.IGNORECASE)
def pick_timestamp_col(columns):
    for c in columns:
        if TS_COL_HINTS.search(str(c)):
            return str(c)
    return ""

DATE_BR = re.compile(r"\b([0-3]?\d)[/.-]([0-1]?\d)[/.-]((?:19|20)\d{2})\b")
def should_use_dayfirst(sample_series: pd.Series) -> bool:
    # Se mais da metade dos valores não nulos parecem dd/mm/yyyy, usar dayfirst=True
    vals = sample_series.dropna().astype(str).head(50).tolist()
    if not vals:
        return False
    hits = sum(1 for v in vals if DATE_BR.search(v))
    return hits >= max(3, len(vals)//2)

def detect_encoding(path: Path, trial=( "utf-8", "latin1", "cp1252" )):
    # tenta abrir poucas linhas em cada encoding
    for enc in trial:
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                _ = f.readline()
            return enc
        except Exception:
            continue
    # fallback permissivo
    return "latin1"

def sniff_delimiter(path: Path, encoding: str):
    try:
        with open(path, "r", encoding=encoding, errors="replace") as f:
            sample = "".join([next(f) for _ in range(20)])
        dialect = csv.Sniffer().sniff(sample, delimiters=",;|\t")
        return dialect.delimiter
    except Exception:
        return ","

def count_lines_fast(path: Path, encoding: str):
    # conta linhas sem carregar tudo na memória
    cnt = 0
    with open(path, "r", encoding=encoding, errors="replace") as f:
        for _ in f:
            cnt += 1
    return cnt

def read_head_tail(path: Path, encoding: str, sep: str, n_head=200, n_tail=200):
    # Lê primeiras N linhas
    try:
        df_head = pd.read_csv(path, encoding=encoding, sep=sep, nrows=n_head, engine="python")
    except Exception:
        df_head = pd.DataFrame()
    # Lê últimas N linhas (precisa saber total de linhas primeiro)
    try:
        total_lines = count_lines_fast(path, encoding)
        # assume 1 linha de header
        data_lines = max(0, total_lines - 1)
        skip = max(1, data_lines - n_tail)  # pular header + início até sobrar tail
        df_tail = pd.read_csv(path, encoding=encoding, sep=sep, skiprows=range(1, 1+skip), engine="python")
    except Exception:
        df_tail = pd.DataFrame()
    return df_head, df_tail

def first_last_timestamp(df_head: pd.DataFrame, df_tail: pd.DataFrame):
    if df_head.empty and df_tail.empty:
        return "", "", "", ""
    cols = df_head.columns if not df_head.empty else df_tail.columns
    ts_col = pick_timestamp_col(cols)
    if not ts_col:
        return "", "", "no", ""
    # decide dayfirst pelo "cheiro" no head
    use_dayfirst = False
    if not df_head.empty and ts_col in df_head.columns:
        use_dayfirst = should_use_dayfirst(df_head[ts_col])
    # parse head
    fts = ""
    lts = ""
    try:
        if not df_head.empty and ts_col in df_head.columns:
            s = pd.to_datetime(df_head[ts_col], errors="coerce", dayfirst=use_dayfirst)
            s = s.dropna()
            if len(s):
                fts = s.iloc[0].isoformat()
    except Exception:
        pass
    # parse tail
    try:
        if not df_tail.empty and ts_col in df_tail.columns:
            s2 = pd.to_datetime(df_tail[ts_col], errors="coerce", dayfirst=use_dayfirst)
            s2 = s2.dropna()
            if len(s2):
                lts = s2.iloc[-1].isoformat()
    except Exception:
        pass
    has_ts = "yes" if (fts or lts) else "uncertain"
    # timespan
    span_days = ""
    try:
        if fts and lts:
            d1 = pd.to_datetime(fts)
            d2 = pd.to_datetime(lts)
            span_days = (d2 - d1).days
    except Exception:
        span_days = ""
    return ts_col, has_ts, fts, lts if lts else "", span_days

# --- Varredura (DRY-RUN, nada salvo) ---

if not BASE_DIR.exists():
    raise FileNotFoundError(f"Pasta base não encontrada: {BASE_DIR}")

files = [p for p in BASE_DIR.rglob("*") if p.is_file()]

rows = []
for p in files:
    rel = str(p.relative_to(ROOT))
    lower = rel.lower()
    ext = p.suffix.lower()
    tguess = ("csv" if ext==".csv" else "xlsx" if ext in (".xlsx",".xls")
              else "txt" if ext==".txt" else "pdf" if ext==".pdf"
              else "md" if ext==".md" else "other")
    fornolist = detect_fornos_anywhere(lower)
    variable_guess = detect_variable_guess(lower)

    info = {
        "file_name": p.name,
        "relative_path": rel,
        "type_guess": tguess,
        "forno_guess": ",".join(fornolist),
        "variable_guess": variable_guess,
        "size_mb": round(p.stat().st_size/(1024*1024),3),
        "last_modified": datetime.fromtimestamp(p.stat().st_mtime).isoformat(timespec="seconds"),
        "n_cols": "",
        "n_rows": "",
        "timestamp_column": "",
        "has_timestamp": "",
        "first_ts": "",
        "last_ts": "",
        "timespan_days": "",
        "encoding_used": "",
    }

    if tguess == "csv":
        enc = detect_encoding(p)
        sep = sniff_delimiter(p, enc)
        info["encoding_used"] = enc

        # n_rows via contagem de linhas (assume header 1 linha)
        try:
            total_lines = count_lines_fast(p, enc)
            info["n_rows"] = max(0, total_lines-1)
        except Exception:
            info["n_rows"] = ""

        # n_cols pelo header
        try:
            df0 = pd.read_csv(p, encoding=enc, sep=sep, nrows=0, engine="python")
            info["n_cols"] = df0.shape[1]
        except Exception:
            info["n_cols"] = ""

        # head/tail para timestamps
        try:
            dfh, dft = read_head_tail(p, enc, sep, n_head=200, n_tail=200)
            ts_col, has_ts, fts, lts, span = first_last_timestamp(dfh, dft)
            info["timestamp_column"] = ts_col
            info["has_timestamp"] = has_ts
            info["first_ts"] = fts
            info["last_ts"] = lts
            info["timespan_days"] = span
        except Exception:
            pass

    rows.append(info)

df = pd.DataFrame(rows)

# matching CSV ↔ dicionários por similaridade de nomes
df["is_dictionary"] = (df["type_guess"]=="xlsx")
df["name_norm"] = df["file_name"].apply(normalize_name_for_match)

dict_df = df[df["is_dictionary"]][["file_name","relative_path","name_norm"]].copy()
data_df = df[~df["is_dictionary"]].copy()

linked, conf = [], []
if dict_df.empty:
    linked = [""]*len(data_df); conf = [0.0]*len(data_df)
else:
    d_names = dict_df["name_norm"].tolist()
    d_files = dict_df["file_name"].tolist()
    for nm in data_df["name_norm"]:
        if not nm:
            linked.append(""); conf.append(0.0); continue
        best_s, best_f = 0.0, ""
        for dn, dfc in zip(d_names, d_files):
            s = similarity(nm, dn)
            if s > best_s:
                best_s, best_f = s, dfc
        linked.append(best_f)
        conf.append(round(100*best_s,1))

data_df["linked_dictionary"] = linked
data_df["link_confidence"] = conf

final_df = pd.concat([data_df.drop(columns=["name_norm"]),
                      df[df["is_dictionary"]].drop(columns=["name_norm"])],
                     ignore_index=True)

final_df = final_df.sort_values(
    by=["is_dictionary","forno_guess","variable_guess","file_name"],
    ascending=[False, True, True, True]
).reset_index(drop=True)

# ---- Relatório em tela (DRY-RUN) ----
print("=== DRY-RUN A1+ (nada salvo) ===")
print(f"Base: {BASE_DIR}")
print("Tipos:")
print(df["type_guess"].value_counts().to_string())
print("\nCobertura de 'forno_guess':")
print(final_df["forno_guess"].value_counts(dropna=False).head(20).to_string())

print("\nPrévia (primeiras 40 linhas):")
display(final_df.head(40)[[
    "file_name","type_guess","forno_guess","variable_guess",
    "n_cols","n_rows","timestamp_column","has_timestamp","first_ts","last_ts","timespan_days",
    "encoding_used","linked_dictionary","link_confidence","size_mb"
]])

print("\nResumo rápido de CSVs com timestamp:")
has_ts = final_df[(final_df["type_guess"]=="csv")]["has_timestamp"].value_counts(dropna=False)
print(has_ts.to_string())


/tmp/ipython-input-592054445.py:163: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  s2 = pd.to_datetime(df_tail[ts_col], errors="coerce", dayfirst=use_dayfirst)
/tmp/ipython-input-592054445.py:163: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  s2 = pd.to_datetime(df_tail[ts_col], errors="coerce", dayfirst=use_dayfirst)
/tmp/ipython-input-592054445.py:163: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  s2 = pd.to_datetime(df_tail[ts_col], errors="coerce", dayfirst=use_dayfirst)
/tmp/ipython-input-592054445.py:163: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  s2 = pd.to_datetime(df_ta

=== DRY-RUN A1+ (nada salvo) ===
Base: /content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados/dados_iniciais
Tipos:
type_guess
csv     124
xlsx      7

Cobertura de 'forno_guess':
forno_guess
F5    28
F4    27
F3    24
F2    24
F1    23
       5

Prévia (primeiras 40 linhas):


,file_name,type_guess,forno_guess,variable_guess,n_cols,n_rows,timestamp_column,has_timestamp,first_ts,last_ts,timespan_days,encoding_used,linked_dictionary,link_confidence,size_mb
0,Dic_eletrodo.xlsx,xlsx,,Comprimento_Eletrodo,,,,,,,,,NaN,NaN,0.064
1,Dic_Consumo.xlsx,xlsx,,Consumo,,,,,,,,,NaN,NaN,0.065
2,Dic_Corrida.xlsx,xlsx,,Corrida,,,,,,,,,NaN,NaN,0.066
3,Dic_Inf_Diário.xlsx,xlsx,,Dicionario_Dados,,,,,,,,,NaN,NaN,0.068
4,Dic_Supervisorio F4.xlsx,xlsx,F4,F4_Supervisorio,,,,,,,,,NaN,NaN,0.223
5,Dic_Supervisorio F5 Hora.xlsx,xlsx,F5,F5_Supervisorio,,,,,,,,,NaN,NaN,0.065
6,Dic_Supervisorio F5 Minuto.xlsx,xlsx,F5,F5_Supervisorio,,,,,,,,,NaN,NaN,0.065
7,Medição Eletrodo.csv,csv,,Comprimento_Eletrodo,5,761,Data medição,yes,2021-01-19T00:00:00,2025-04-25T00:00:00,1557,latin1,Dic_eletrodo.xlsx,80.0,0.020
8,2018_F1_Inf.Diario.csv,csv,F1,,52,365,Data_base,yes,2018-01-01T00:00:00,2018-12-31T00:00:00,364,utf-8,Dic_Inf_Diário.xlsx,66.7,0.088
9,2019_F1_Inf.Diario.csv,csv,F1,,45,365,Data_base,yes,2019-01-01T00:00:00,2019-12-31T00:00:00,364,utf-8,Dic_Inf_Diário.xlsx,66.7,0.084



Resumo rápido de CSVs com timestamp:
has_timestamp
yes    123
         1


In [12]:
# CÉLULA A2 — Persistente: gera manifest_index.csv + readme_manifest.md SEM warnings de data

from pathlib import Path
import os, re, io, csv, hashlib
from datetime import datetime
from difflib import SequenceMatcher
import pandas as pd

# Caminhos
ROOT = Path("/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados")
BASE_DIR = ROOT / "dados_iniciais"
OUT_DIR  = ROOT / "manifestos_final"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_CSV  = OUT_DIR / "manifest_index.csv"
OUT_MD   = OUT_DIR / "readme_manifest.md"

# ---------- Heurísticas (iguais às DRY-RUN, com forno mais robusto) ----------

FORNO_REGEXES = [
    re.compile(r"(?i)\bF\s*[-_ ]?\s*(\d{1,2})\b"),
    re.compile(r"(?i)F\s*[-_ ]?\s*(\d{1,2})"),
    re.compile(r"(?i)\bForno\s*[-_ ]?\s*(\d{1,2})\b")
]
def detect_fornos_anywhere(text: str):
    found = []
    for rgx in FORNO_REGEXES:
        for m in rgx.findall(text):
            num = str(m) if isinstance(m, (str,int)) else (m[0] if m else "")
            if num:
                tag = f"F{int(num)}"
                if tag not in found:
                    found.append(tag)
    return found

VAR_KEYWORDS = [
    (r"consum", "Consumo"),
    (r"corrid|corrida|run", "Corrida"),
    (r"granul|granulo", "Granulometria"),
    (r"umidad", "Umidade"),
    (r"poten|kw|power", "Potencia"),
    (r"corrente|amp|amper", "Corrente"),
    (r"tens[aã]o|volt", "Tensao"),
    (r"carg|charge|aliment", "Carga/Alimentacao"),
    (r"temp|temperat", "Temperatura"),
    (r"rendi|yield", "Rendimento"),
    (r"eletrod|compriment|eletrodo", "Comprimento_Eletrodo"),
    (r"supervis", "Supervisorio"),
    (r"\bf4\b", "F4_Supervisorio"),
    (r"\bf5\b", "F5_Supervisorio"),
    (r"dicion|data[_-]?dict|dic\b", "Dicionario_Dados"),
]
def detect_variable_guess(text: str):
    t = text.lower()
    hits = []
    for pat, label in VAR_KEYWORDS:
        if re.search(pat, t):
            hits.append(label)
    if not hits:
        return ""
    order = ["F4_Supervisorio","F5_Supervisorio","Supervisorio","Comprimento_Eletrodo",
             "Consumo","Corrida","Temperatura","Potencia","Corrente","Tensao",
             "Carga/Alimentacao","Granulometria","Umidade","Rendimento","Dicionario_Dados"]
    for lbl in order:
        if lbl in hits:
            return lbl
    return hits[0]

def normalize_name_for_match(name: str):
    base = os.path.splitext(name)[0].lower()
    base = re.sub(r"[\W_]+", " ", base)
    base = re.sub(r"\b(v\d+|vers[aã]o?\d*|final|novo|corrigido)\b", " ", base)
    base = re.sub(r"\s+", " ", base).strip()
    return base

def similarity(a: str, b: str) -> float:
    return SequenceMatcher(None, a, b).ratio()

# ---------- Utilidades CSV ----------

def detect_encoding(path: Path, trial=("utf-8","latin1","cp1252")):
    for enc in trial:
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                _ = f.readline()
            return enc
        except Exception:
            continue
    return "latin1"

def sniff_delimiter(path: Path, encoding: str):
    try:
        with open(path, "r", encoding=encoding, errors="replace") as f:
            sample = "".join([next(f) for _ in range(20)])
        dialect = csv.Sniffer().sniff(sample, delimiters=",;|\t")
        return dialect.delimiter
    except Exception:
        return ","

def count_lines_fast(path: Path, encoding: str):
    cnt = 0
    with open(path, "r", encoding=encoding, errors="replace") as f:
        for _ in f:
            cnt += 1
    return cnt

def read_head_tail(path: Path, encoding: str, sep: str, n_head=200, n_tail=200):
    try:
        df_head = pd.read_csv(path, encoding=encoding, sep=sep, nrows=n_head, engine="python")
    except Exception:
        df_head = pd.DataFrame()
    try:
        total_lines = count_lines_fast(path, encoding)
        data_lines = max(0, total_lines - 1)  # exclui header
        skip = max(1, data_lines - n_tail)
        df_tail = pd.read_csv(path, encoding=encoding, sep=sep, skiprows=range(1, 1+skip), engine="python")
    except Exception:
        df_tail = pd.DataFrame()
    return df_head, df_tail

# ---------- Datas: detecção de estilo + parse sem warnings ----------

BR_RE  = re.compile(r"\b([0-3]?\d)[/.-]([0-1]?\d)[/.-]((?:19|20)\d{2})\b")  # dd/mm/yyyy
US_RE  = re.compile(r"\b([0-1]?\d)[/.-]([0-3]?\d)[/.-]((?:19|20)\d{2})\b")  # mm/dd/yyyy
ISO_RE = re.compile(r"\b(20\d{2}|19\d{2})[-/](0[1-9]|1[0-2])[-/](0[1-9]|[12]\d|3[01])\b") # yyyy-mm-dd

def detect_ts_col(columns):
    ts_pat = re.compile(r"(date|data|time|hora|timestamp|dt|datetime)", re.IGNORECASE)
    for c in columns:
        if ts_pat.search(str(c)):
            return str(c)
    return ""

def guess_date_style(series: pd.Series):
    vals = series.dropna().astype(str).head(80).tolist()
    if not vals:
        return "unknown"
    br = sum(1 for v in vals if BR_RE.search(v))
    us = sum(1 for v in vals if US_RE.search(v))
    iso = sum(1 for v in vals if ISO_RE.search(v) or ("-" in v and ":" in v))  # tolera datetime ISO
    # decisão
    if max(br, us, iso) == 0:
        return "unknown"
    if iso >= br and iso >= us:
        return "iso"
    if br > us:
        return "br"
    if us > br:
        return "us"
    # empate: marcar mixed
    return "mixed"

def parse_dates_series(s: pd.Series, style: str):
    # Evita warnings definindo format quando possível
    if style == "br":
        try:
            return pd.to_datetime(s, format="%d/%m/%Y", errors="coerce")
        except Exception:
            pass
        return pd.to_datetime(s, dayfirst=True, errors="coerce")
    elif style == "us":
        try:
            return pd.to_datetime(s, format="%m/%d/%Y", errors="coerce")
        except Exception:
            pass
        return pd.to_datetime(s, dayfirst=False, errors="coerce")
    elif style == "iso":
        # ISO costuma parsear direto, sem warnings
        return pd.to_datetime(s, errors="coerce")
    elif style == "mixed":
        # tenta BR e US e escolhe o que tiver mais sucesso
        br_parsed = pd.to_datetime(s, errors="coerce", dayfirst=True)
        us_parsed = pd.to_datetime(s, errors="coerce", dayfirst=False)
        if br_parsed.notna().sum() >= us_parsed.notna().sum():
            return br_parsed
        return us_parsed
    else:
        # unknown → tenta ISO/auto (sem dayfirst explícito)
        return pd.to_datetime(s, errors="coerce")

def first_last_from_two(df_head: pd.DataFrame, df_tail: pd.DataFrame):
    if df_head.empty and df_tail.empty:
        return "", "", "", "", ""
    cols = df_head.columns if not df_head.empty else df_tail.columns
    ts_col = detect_ts_col(cols)
    if not ts_col:
        return "", "no", "", "", ""
    # detecta estilo com base no head
    style = "unknown"
    if not df_head.empty and ts_col in df_head.columns:
        style = guess_date_style(df_head[ts_col])

    fts, lts = "", ""
    # parse head
    if not df_head.empty and ts_col in df_head.columns:
        s = parse_dates_series(df_head[ts_col], style).dropna()
        if len(s):
            fts = s.iloc[0].isoformat()
    # parse tail
    if not df_tail.empty and ts_col in df_tail.columns:
        s2 = parse_dates_series(df_tail[ts_col], style).dropna()
        if len(s2):
            lts = s2.iloc[-1].isoformat()

    has_ts = "yes" if (fts or lts) else "uncertain"
    span = ""
    try:
        if fts and lts:
            d1, d2 = pd.to_datetime(fts), pd.to_datetime(lts)
            span = (d2 - d1).days
    except Exception:
        span = ""
    return ts_col, has_ts, fts, lts, span

# ---------- Varredura e construção do manifesto ----------

if not BASE_DIR.exists():
    raise FileNotFoundError(f"Pasta base não encontrada: {BASE_DIR}")

files = [p for p in BASE_DIR.rglob("*") if p.is_file()]

rows = []
for p in files:
    rel = str(p.relative_to(ROOT))
    lower = rel.lower()
    ext = p.suffix.lower()
    tguess = ("csv" if ext==".csv" else "xlsx" if ext in (".xlsx",".xls")
              else "txt" if ext==".txt" else "pdf" if ext==".pdf"
              else "md" if ext==".md" else "other")
    fornolist = detect_fornos_anywhere(lower)
    variable_guess = detect_variable_guess(lower)

    item = {
        "file_name": p.name,
        "relative_path": rel,
        "ext": ext,
        "type_guess": tguess,
        "forno_guess": ",".join(fornolist),
        "variable_guess": variable_guess,
        "size_mb": round(p.stat().st_size/(1024*1024),3),
        "last_modified": datetime.fromtimestamp(p.stat().st_mtime).isoformat(timespec="seconds"),
        "n_cols": "",
        "n_rows": "",
        "timestamp_column": "",
        "has_timestamp": "",
        "first_ts": "",
        "last_ts": "",
        "timespan_days": "",
        "encoding_used": "",
        "linked_dictionary": "",
        "link_confidence": ""
    }

    if tguess == "csv":
        enc = detect_encoding(p)
        sep = sniff_delimiter(p, enc)
        item["encoding_used"] = enc

        # n_rows e n_cols
        try:
            total_lines = count_lines_fast(p, enc)
            item["n_rows"] = max(0, total_lines-1)
        except Exception:
            pass
        try:
            df0 = pd.read_csv(p, encoding=enc, sep=sep, nrows=0, engine="python")
            item["n_cols"] = df0.shape[1]
        except Exception:
            pass

        # head/tail para timestamps (agora sem warnings)
        try:
            dfh, dft = read_head_tail(p, enc, sep, n_head=200, n_tail=200)
            ts_col, has_ts, fts, lts, span = first_last_from_two(dfh, dft)
            item["timestamp_column"] = ts_col
            item["has_timestamp"]     = has_ts
            item["first_ts"]          = fts
            item["last_ts"]           = lts
            item["timespan_days"]     = span
        except Exception:
            pass

    rows.append(item)

df = pd.DataFrame(rows)

# Matching CSV ↔ XLSX (dicionários)
df["is_dictionary"] = (df["type_guess"]=="xlsx")
df["name_norm"] = df["file_name"].apply(normalize_name_for_match)

dict_df = df[df["is_dictionary"]][["file_name","relative_path","name_norm"]].copy()
data_df = df[~df["is_dictionary"]].copy()

linked, conf = [], []
if dict_df.empty:
    linked = [""]*len(data_df)
    conf   = [0.0]*len(data_df)
else:
    d_names = dict_df["name_norm"].tolist()
    d_files = dict_df["file_name"].tolist()
    for nm in data_df["name_norm"]:
        if not nm:
            linked.append(""); conf.append(0.0); continue
        best_s, best_f = 0.0, ""
        for dn, dfc in zip(d_names, d_files):
            s = similarity(nm, dn)
            if s > best_s:
                best_s, best_f = s, dfc
        linked.append(best_f)
        conf.append(round(100*best_s,1))

data_df["linked_dictionary"] = linked
data_df["link_confidence"]   = conf

final_df = pd.concat([
    data_df.drop(columns=["name_norm"]),
    df[df["is_dictionary"]].drop(columns=["name_norm"])
], ignore_index=True)

final_df = final_df.sort_values(
    by=["is_dictionary","forno_guess","variable_guess","file_name"],
    ascending=[False, True, True, True]
).reset_index(drop=True)

# ---------- Salvar CSV + README ----------

final_df.to_csv(OUT_CSV, index=False, encoding="utf-8")

readme = []
readme.append("# Manifesto de dados — versão persistente")
readme.append("")
readme.append("Este manifesto cataloga os arquivos em `dados_iniciais`, com colunas:")
readme.append("- file_name, relative_path, ext, type_guess, size_mb, last_modified")
readme.append("- forno_guess (detecção robusta, pega F* em qualquer posição do caminho)")
readme.append("- variable_guess (palavras-chave de domínio)")
readme.append("- n_cols, n_rows (para CSV)")
readme.append("- timestamp_column, has_timestamp, first_ts, last_ts, timespan_days (parse sem warnings via detecção BR/US/ISO)")
readme.append("- encoding_used (para CSV)")
readme.append("- linked_dictionary, link_confidence (matching por similaridade ao XLSX mais próximo)")
with open(OUT_MD, "w", encoding="utf-8") as f:
    f.write("\n".join(readme))

print("Concluído sem warnings de data.")
print("Saídas gravadas em:")
print("-", OUT_CSV)
print("-", OUT_MD)

print("\nResumo rápido:")
print("Tipos:\n", final_df["type_guess"].value_counts().to_string())
print("\nCobertura de forno_guess (top 10):\n", final_df["forno_guess"].value_counts(dropna=False).head(10).to_string())
print("\nCSVs com timestamp:\n", final_df[final_df["type_guess"]=="csv"]["has_timestamp"].value_counts(dropna=False).to_string())


/tmp/ipython-input-2370670043.py:171: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  br_parsed = pd.to_datetime(s, errors="coerce", dayfirst=True)
/tmp/ipython-input-2370670043.py:171: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  br_parsed = pd.to_datetime(s, errors="coerce", dayfirst=True)
/tmp/ipython-input-2370670043.py:171: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  br_parsed = pd.to_datetime(s, errors="coerce", dayfirst=True)
/tmp/ipython-input-2370670043.py:171: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  br_parsed = pd.to_datetime(s, errors="coerce", dayfirst=True)
/tmp

Concluído sem warnings de data.
Saídas gravadas em:
- /content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados/manifestos_final/manifest_index.csv
- /content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados/manifestos_final/readme_manifest.md

Resumo rápido:
Tipos:
 type_guess
csv     124
xlsx      7

Cobertura de forno_guess (top 10):
 forno_guess
F5    28
F4    27
F3    24
F2    24
F1    23
       5

CSVs com timestamp:
 has_timestamp
yes    123
         1


In [14]:
# Conversão do Manifesto CSV para Markdown compatível com NotebookLM
from pathlib import Path
import pandas as pd
import textwrap

# Caminhos
csv_path = Path("/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados/manifestos_final/manifest_index.csv")
md_path  = csv_path.with_suffix(".md")

# Carregar CSV
df = pd.read_csv(csv_path)

# Criar cabeçalho explicativo
header = textwrap.dedent("""
# Manifesto de Dados — Fase 1 Diagnóstico (NotebookLM)

> Este documento é uma versão **Markdown** do `manifest_index.csv` para uso no NotebookLM.
> Cada linha representa um arquivo disponível em `dados_iniciais`.

**Colunas principais**
- `file_name` – nome do arquivo
- `relative_path` – caminho relativo completo
- `type_guess` – tipo detectado (csv, xlsx, pdf…)
- `forno_guess` – forno detectado (F1–F5)
- `variable_guess` – variável principal inferida (Consumo, Corrida, etc.)
- `n_cols`, `n_rows` – número de colunas e linhas (para CSVs)
- `timestamp_column`, `first_ts`, `last_ts`, `timespan_days` – informações temporais
- `linked_dictionary` – dicionário de dados associado
- `link_confidence` – confiança da correspondência (%)
- `size_mb`, `last_modified`, `encoding_used`

---
""").strip()

# Converter a tabela para markdown (máx. 1000 linhas para evitar excesso)
preview_df = df.head(1000)  # limite opcional; retire se quiser o manifesto completo
table_md = preview_df.to_markdown(index=False)

# Concatenar e salvar
content = header + "\n\n" + table_md + "\n"
md_path.write_text(content, encoding="utf-8")

print(f"✅ Markdown criado com sucesso!\n\nLocal:\n{md_path}")


✅ Markdown criado com sucesso!

Local:
/content/drive/MyDrive/GRUPO MARINGÁ/FASE 1 - DIAGNÓSTICO/Dados/manifestos_final/manifest_index.md
